# Axon ML Models - Model Registry

This notebook trains ML models for the Axon Intelligence Agent:
- **Evidence Upload Volume Forecasting** - Predict future monthly evidence_upload_volume
- **Agency Churn Prediction** - Classify agencies at risk of churning
- **Device Deployment Success** - Predict which device deployments will be successful

All models are registered to Snowflake Model Registry and can be added as tools to the Intelligence Agent.

## Prerequisites

**Required Packages** (configured automatically):
- `snowflake-ml-python`
- `scikit-learn`
- `xgboost`
- `matplotlib`

**Database Context:**
- **Database:** AXON_INTELLIGENCE  
- **Schema:** ANALYTICS  
- **Warehouse:** AXON_WH

**Note:** This notebook uses Snowflake Model Registry. Ensure you have appropriate permissions to create and register models.


## Import Required Packages


In [ ]:
# Import Python packages
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Import Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

# Import Snowpark ML
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LinearRegression, LogisticRegression
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_error, accuracy_score, roc_auc_score
from snowflake.ml.registry import Registry

print("✅ Packages imported successfully")


## Connect to Snowflake

Get active session and set context to Axon database.


In [ ]:
# Get active Snowflake session
session = get_active_session()

# Set context
session.use_database('AXON_INTELLIGENCE')
session.use_schema('ANALYTICS')
session.use_warehouse('AXON_WH')

print(f"✅ Connected - Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Database.Schema: {session.get_fully_qualified_current_schema()}")


---
# MODEL 1: Evidence Upload Volume Forecasting

Predict future monthly evidence_upload_volume using historical order data.


### Prepare Revenue Training Data


In [ ]:
# Get monthly evidence upload volume data with features
evidence_upload_volume_df = session.sql("""
SELECT
    DATE_TRUNC('month', upload_date)::DATE AS upload_month,
    MONTH(upload_date) AS month_num,
    YEAR(upload_date) AS year_num,
    COUNT(DISTINCT evidence_id)::FLOAT AS total_evidence_upload_volume,
    COUNT(DISTINCT deployment_id)::FLOAT AS deployment_count,
    COUNT(DISTINCT agency_id)::FLOAT AS agency_count,
    AVG(file_size_mb)::FLOAT AS avg_file_size_mb
FROM RAW.EVIDENCE_UPLOADS
WHERE upload_date >= DATEADD('month', -30, CURRENT_DATE())
  AND evidence_status = 'ACTIVE'
GROUP BY DATE_TRUNC('month', upload_date), MONTH(upload_date), YEAR(upload_date)
ORDER BY upload_month
""")

print(f"Evidence upload volume data: {evidence_upload_volume_df.count()} months")
evidence_upload_volume_df.show(5)


### Split Data and Train Revenue Model


In [ ]:
# Train/test split (last 6 months for testing)
train_evidence_upload_volume = evidence_upload_volume_df.filter(F.col("UPLOAD_MONTH") < F.dateadd("month", F.lit(-6), F.current_date()))
test_evidence_upload_volume = evidence_upload_volume_df.filter(F.col("UPLOAD_MONTH") >= F.dateadd("month", F.lit(-6), F.current_date()))

# Drop UPLOAD_MONTH (DATE type not supported in pipeline)
train_evidence_upload_volume = train_evidence_upload_volume.drop("UPLOAD_MONTH")
test_evidence_upload_volume = test_evidence_upload_volume.drop("UPLOAD_MONTH")

# Create pipeline
evidence_upload_volume_pipeline = Pipeline([
    ("Scaler", StandardScaler(
        input_cols=["MONTH_NUM", "DEPLOYMENT_COUNT", "AGENCY_COUNT", "AVG_FILE_SIZE_MB"],
        output_cols=["MONTH_NUM_SCALED", "DEPLOYMENT_COUNT_SCALED", "AGENCY_COUNT_SCALED", "AVG_FILE_SIZE_MB_SCALED"]
    )),
    ("LinearRegression", LinearRegression(
        label_cols=["TOTAL_EVIDENCE_UPLOAD_VOLUME"],
        output_cols=["PREDICTED_EVIDENCE_UPLOAD_VOLUME"]
    ))
])

# Train model
evidence_upload_volume_pipeline.fit(train_evidence_upload_volume)
print("✅ Evidence upload volume forecasting model trained")


### Evaluate and Register Revenue Model


In [ ]:
# Make predictions on test set
test_predictions = evidence_upload_volume_pipeline.predict(test_evidence_upload_volume)

# Calculate metrics
mae = mean_absolute_error(df=test_predictions, y_true_col_names="TOTAL_EVIDENCE_UPLOAD_VOLUME", y_pred_col_names="PREDICTED_EVIDENCE_UPLOAD_VOLUME")
mse = mean_squared_error(df=test_predictions, y_true_col_names="TOTAL_EVIDENCE_UPLOAD_VOLUME", y_pred_col_names="PREDICTED_EVIDENCE_UPLOAD_VOLUME")
rmse = mse ** 0.5

metrics = {"mae": round(mae, 2), "rmse": round(rmse, 2)}
print(f"Model metrics: {metrics}")

# Register model
reg = Registry(session)
reg.log_model(
    model=evidence_upload_volume_pipeline,
    model_name="EVIDENCE_VOLUME_PREDICTOR",
    version_name="V1",
    comment="Predicts monthly evidence upload volume based on historical deployment and storage patterns using Linear Regression",
    metrics=metrics
)

print("✅ Evidence volume model registered to Model Registry as EVIDENCE_VOLUME_PREDICTOR")


---
# MODEL 2: Agency Churn Prediction

Classify agencies as likely to churn or not based on behavior patterns.


### Prepare Churn Training Data


In [ ]:
# Get agency features for churn prediction
churn_df = session.sql("""
SELECT
    a.agency_id,
    a.agency_type AS agency_segment,
    a.jurisdiction_type AS jurisdiction,
    a.lifetime_value::FLOAT AS lifetime_value,
    a.population_served::FLOAT AS population_served,
    -- Recent orders (last 3 months)
    COUNT(DISTINCT CASE WHEN o.order_date >= DATEADD('month', -3, CURRENT_DATE()) 
                   THEN o.order_id END)::FLOAT AS recent_orders,
    -- Historical average
    (COUNT(DISTINCT CASE WHEN o.order_date < DATEADD('month', -3, CURRENT_DATE()) 
                    THEN o.order_id END) / 9.0)::FLOAT AS historical_avg_orders,
    -- Support satisfaction
    AVG(CASE WHEN st.created_date >= DATEADD('month', -6, CURRENT_DATE()) 
        THEN st.customer_satisfaction_score::FLOAT END) AS avg_csat,
    -- Quality issues
    COUNT(DISTINCT qi.quality_issue_id)::FLOAT AS quality_issue_count,
    -- Device deployments
    COUNT(DISTINCT CASE WHEN dd.deployment_date >= DATEADD('month', -12, CURRENT_DATE()) 
                   THEN dd.deployment_id END)::FLOAT AS recent_device_deployments,
    -- Target: Is churned (simplified logic)
    (a.agency_status = 'CHURNED')::BOOLEAN AS is_churned
FROM RAW.AGENCIES a
LEFT JOIN RAW.ORDERS o ON a.agency_id = o.agency_id
LEFT JOIN RAW.SUPPORT_TICKETS st ON a.agency_id = st.agency_id
LEFT JOIN RAW.QUALITY_ISSUES qi ON a.agency_id = qi.agency_id
LEFT JOIN RAW.DEVICE_DEPLOYMENTS dd ON a.agency_id = dd.agency_id
WHERE a.agency_status IN ('ACTIVE', 'CHURNED')  -- Only include active or churned agencies
GROUP BY a.agency_id, a.agency_type, a.jurisdiction_type, a.lifetime_value, a.population_served, a.agency_status
HAVING COUNT(DISTINCT o.order_id) > 0  -- Changed from 10 to 0 (any orders)
   OR COUNT(DISTINCT dd.deployment_id) > 0  -- OR has any deployments
LIMIT 5000  -- Limit to 5000 agencies for faster training
""")

print(f"Churn data: {churn_df.count()} agencies")
churn_df.show(5)


### Train Churn Classification Model


In [ ]:
# Train/test split (80/20)
train_churn, test_churn = churn_df.random_split([0.8, 0.2], seed=42)

# Drop AGENCY_ID
train_churn = train_churn.drop("AGENCY_ID")
test_churn = test_churn.drop("AGENCY_ID")

# Create pipeline with preprocessing and classification
churn_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["AGENCY_SEGMENT", "JURISDICTION"],
        output_cols=["AGENCY_SEGMENT_ENCODED", "JURISDICTION_ENCODED"],
        drop_input_cols=True,  # Drop original string columns after encoding
        handle_unknown="ignore"
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["IS_CHURNED"],
        output_cols=["CHURN_PREDICTION"],
        n_estimators=100,
        max_depth=10
    ))
])

# Train model
churn_pipeline.fit(train_churn)
print("✅ Churn classification model trained")


### Evaluate and Register Churn Model


In [ ]:
# Make predictions
churn_predictions = churn_pipeline.predict(test_churn)

# Calculate metrics
accuracy = accuracy_score(df=churn_predictions, y_true_col_names="IS_CHURNED", y_pred_col_names="CHURN_PREDICTION")
# Note: ROC AUC might need probability scores - using accuracy for now
churn_metrics = {"accuracy": round(accuracy, 4)}
print(f"Churn model metrics: {churn_metrics}")

# Register model (use different name to avoid conflict)
reg.log_model(
    model=churn_pipeline,
    model_name="AGENCY_CHURN_PREDICTOR",
    version_name="V1",
    comment="Predicts customer churn probability using Random Forest based on behavior patterns",
    metrics=churn_metrics
)

print("✅ Churn model registered to Model Registry as AGENCY_CHURN_PREDICTOR")


---
# MODEL 3: Device Deployment Success Prediction

Predict which design wins are likely to convert to production orders.


### Prepare Device Deployment Success Data


In [ ]:
# Get device deployment features
deployment_success_df = session.sql("""
SELECT
    dd.deployment_id,
    p.product_family,
    a.agency_type AS agency_segment,
    a.jurisdiction_type AS jurisdiction,
    a.population_served::FLOAT AS population_served,
    dd.competitive_replacement::BOOLEAN AS is_competitive_win,
    o.officer_status,
    o.axon_certified::BOOLEAN AS officer_certified,
    -- Deployment success: has evidence uploads and still active
    (dd.deployment_status = 'ACTIVE' 
     AND EXISTS (SELECT 1 FROM RAW.EVIDENCE_UPLOADS ev 
                 WHERE ev.deployment_id = dd.deployment_id))::BOOLEAN AS deployment_successful
FROM RAW.DEVICE_DEPLOYMENTS dd
JOIN RAW.PRODUCT_CATALOG p ON dd.product_id = p.product_id
JOIN RAW.AGENCIES a ON dd.agency_id = a.agency_id
JOIN RAW.OFFICERS o ON dd.officer_id = o.officer_id
WHERE dd.deployment_date >= DATEADD('month', -24, CURRENT_DATE())
""")

print(f"Deployment data: {deployment_success_df.count()} deployments")
deployment_success_df.show(5)


### Train Conversion Model


In [ ]:
# Split data
train_deployment_success, test_deployment_success = deployment_success_df.random_split([0.8, 0.2], seed=42)

# Drop DEPLOYMENT_ID (VARCHAR not supported as feature)
train_deployment_success = train_deployment_success.drop("DEPLOYMENT_ID")
test_deployment_success = test_deployment_success.drop("DEPLOYMENT_ID")

# Create pipeline
deployment_success_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["PRODUCT_FAMILY", "AGENCY_SEGMENT", "JURISDICTION", "OFFICER_STATUS"],
        output_cols=["PRODUCT_FAMILY_ENC", "AGENCY_SEGMENT_ENC", "JURISDICTION_ENC", "OFFICER_STATUS_ENC"],
        drop_input_cols=True,  # Drop original string columns after encoding
        handle_unknown="ignore"
    )),
    ("Classifier", LogisticRegression(
        label_cols=["DEPLOYMENT_SUCCESSFUL"],
        output_cols=["SUCCESS_PREDICTION"]
    ))
])

# Train
deployment_success_pipeline.fit(train_deployment_success)
print("✅ Deployment success model trained")


### Evaluate and Register Conversion Model


In [ ]:
# Predict on test set
deployment_success_predictions = deployment_success_pipeline.predict(test_deployment_success)

# Calculate accuracy
success_accuracy = accuracy_score(df=deployment_success_predictions, 
                                   y_true_col_names="DEPLOYMENT_SUCCESSFUL",
                                   y_pred_col_names="SUCCESS_PREDICTION")
success_metrics = {"accuracy": round(success_accuracy, 4)}
print(f"Deployment success model metrics: {success_metrics}")

# Register model
reg.log_model(
    model=deployment_success_pipeline,
    model_name="DEPLOYMENT_SUCCESS_PREDICTOR",
    version_name="V1",
    comment="Predicts deployment success (active with evidence uploads) using Logistic Regression based on officer, agency, and product features",
    metrics=success_metrics
)

print("✅ Deployment success model registered to Model Registry as DEPLOYMENT_SUCCESS_PREDICTOR")


---
# Verify Models in Registry


In [ ]:
# Show all models in the registry
print("Models in registry:")
reg.show_models()

# Show versions for evidence volume model
print("\nEvidence Volume Predictor versions:")
reg.get_model("EVIDENCE_VOLUME_PREDICTOR").show_versions()

# Show versions for churn model  
print("\nAgency Churn Predictor versions:")
reg.get_model("AGENCY_CHURN_PREDICTOR").show_versions()

# Show versions for deployment success model
print("\nDeployment Success Predictor versions:")
reg.get_model("DEPLOYMENT_SUCCESS_PREDICTOR").show_versions()

print("\n✅ All models registered and ready to add to Intelligence Agent")


---
# Test Model Inference

Test calling each model to make predictions.


In [ ]:
# Test evidence volume forecast on recent data
evidence_upload_volume_model = reg.get_model("EVIDENCE_VOLUME_PREDICTOR").default
recent_evidence_upload_volume = evidence_upload_volume_df.limit(3).drop("UPLOAD_MONTH")
evidence_upload_volume_preds = evidence_upload_volume_model.run(recent_evidence_upload_volume, function_name="predict")
print("Evidence Volume predictions:")
evidence_upload_volume_preds.select("TOTAL_EVIDENCE_UPLOAD_VOLUME", "PREDICTED_EVIDENCE_UPLOAD_VOLUME").show()

# Test churn prediction on sample agencies
churn_model = reg.get_model("AGENCY_CHURN_PREDICTOR").default
sample_agencies = churn_df.limit(5).drop("AGENCY_ID")
churn_preds = churn_model.run(sample_agencies, function_name="predict")
print("\nChurn predictions:")
churn_preds.select("IS_CHURNED", "CHURN_PREDICTION").show()

# Test deployment success prediction
deployment_success_model = reg.get_model("DEPLOYMENT_SUCCESS_PREDICTOR").default
sample_deployments = deployment_success_df.limit(5).drop("DEPLOYMENT_ID")
deployment_success_preds = deployment_success_model.run(sample_deployments, function_name="predict")
print("\nDeployment Success predictions:")
deployment_success_preds.select("DEPLOYMENT_SUCCESSFUL", "SUCCESS_PREDICTION").show()

print("\n✅ All models tested successfully!")


---
# Next Steps

## Add Models to Intelligence Agent

**Option 1: Using the SQL Script (Easiest)**
Run `sql/agent/08_create_intelligence_agent.sql` which automatically configures all 3 ML models.

**Option 2: Manual Configuration in Snowsight**
1. In Snowsight → AI & ML → Agents → AXON_INTELLIGENCE_AGENT
2. Go to Tools → + Add → Function
3. Add each model wrapper procedure:
   - **PREDICT_EVIDENCE_UPLOAD_VOLUME** (from `sql/ml/07_create_model_wrapper_functions.sql`)
   - **PREDICT_AGENCY_CHURN** (from `sql/ml/07_create_model_wrapper_functions.sql`)
   - **PREDICT_DEPLOYMENT_SUCCESS** (from `sql/ml/07_create_model_wrapper_functions.sql`)

## Example Questions for Agent

- "Predict evidence upload volume for the next 6 months"
- "Which agencies are at high risk of churn?"
- "What is the predicted success rate for deploying Body Camera 3 to Officer OFC00012345?"
- "Forecast storage needs for Evidence.com over the next quarter"

The models will now be available as tools your agent can use!
